In [1]:
f1 = open('user_5k_avg', 'r').read()
user_avg = eval(f1)

f2 = open('business_5k_avg', 'r').read()
business_avg = eval(f2)

f3 = open('review_5k_user', 'r').read()
review5k_user = eval(f3)

f4 = open('review_5k_business', 'r').read()
review5k_business = eval(f4)

f5 = open('review_5k_rating', 'r').read()
review5k_rating = eval(f5)

f6 = open('review_5k_text', 'r').read()
review5k_text = eval(f6)

f7 = open('relation_5k', 'r').read()
relation = eval(f7)


In [2]:
import random 
import math
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer


<h1>Parameters</h1>

In [3]:
random_test = random.sample(xrange(len(review5k_rating)), 1000)

train_user = []
train_business = []
train_rating = []
train_text = []

test_user = []
test_business = []
test_rating = []

for r in xrange(len(review5k_rating)):
    if r in random_test:
        test_user.append(review5k_user[r])
        test_business.append(review5k_business[r])
        test_rating.append(review5k_rating[r])
    else:
        train_user.append(review5k_user[r])
        train_business.append(review5k_business[r])
        train_rating.append(review5k_rating[r])
        train_text.append(review5k_text[r])
        
K_topic = 10
Times = 50
DocWord = 300
DocTopic = 5

sim_val = 0.85 # default = 0.85
vip_val = 0.2
lbd1 = 0.5 # Similarity effect 
lbd2 = 0.5 # relation afftect
lbd3 = 0.5 # VIP effect

<h1>Initialization</h1>

In [4]:
num_user = len(user_avg)
num_business = len(business_avg)
num_train = len(train_rating)
num_test = len(test_rating)

mu = np.mean(train_rating)

def float2int(f):
    if f < 1.5: return 1
    if f < 2.5: return 2
    if f < 3.5: return 3
    if f < 4.5: return 4
    return 5

tokenizer = RegexpTokenizer(r'\w+')
en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

def prep(doc):
    raw = doc.lower().replace("\n", "").replace("\t", "")
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    return (" ").join(texts)

from sklearn import linear_model
def MLR(X, Y):
    reg = linear_model.LinearRegression()
    reg.fit(X, Y)
    return reg.coef_

<h1>Basic Model</h1>

In [5]:
Ubb = []
for i in xrange(num_user):
    Ubb.append([])
    for j in xrange(num_business):
        val = user_avg[i] + business_avg[j] - mu
        if val < 1: val = 1
        if val > 5: val = 5
        Ubb[i].append(val) 

In [6]:
UbbPd = []
for r in xrange(num_test):
    UbbPd.append(Ubb[test_user[r]][test_business[r]]) 

#UbbPdInt = []
#for r in UbbPd: 
#    UbbPdInt.append(float2int(r)) 

Ubb_rmse = mean_squared_error(test_rating, UbbPd)  
#UbbPdInt_rmse = mean_squared_error(test_rating, UbbPdInt) 

print Ubb_rmse
#print UbbPdInt_rmse

0.772537289011


In [7]:
BasicIn = []
for i in xrange(num_user):
    BasicIn.append([])
    for j in xrange(num_business):
        val = user_avg[i] + business_avg[j] - mu
        if val < 1: val = 1
        if val > 5: val = 5
        BasicIn[i].append(val) 
        
for r in xrange(num_train):
    BasicIn[train_user[r]][train_business[r]] = train_rating[r]


In [8]:
model = NMF(n_components=K_topic, init='random', random_state=0)
U = model.fit_transform(BasicIn);
V = model.components_;
BasicOut = np.dot(U,V)

In [9]:
BasicPd = []
for r in xrange(num_test):
    BasicPd.append(BasicOut[test_user[r]][test_business[r]]) 

#BasicPdInt = []
#for r in BasicPd: 
#    BasicPdInt.append(float2int(r)) 

BasicPd_rmse = mean_squared_error(test_rating, BasicPd)  
#BasicPdInt_rmse = mean_squared_error(test_rating, BasicPdInt) 

print BasicPd_rmse
#print BasicPdInt_rmse

0.764162010064


<h1>Topic Model</h1>

In [10]:
Breview = [""]*num_business
for r in xrange(num_train):
    Breview[train_business[r]] += prep(train_text[r])

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=DocWord, stop_words='english')
tf = tf_vectorizer.fit_transform(Breview)

lda = LatentDirichletAllocation(n_topics=DocTopic, max_iter=5, learning_method='online',learning_offset=50.,random_state=0)
DocTopDist = lda.fit_transform(tf)


In [11]:
row = np.array([])
col = np.array([])
val = np.array([])

# delta -> difference between rating and ubb [u*b], delta2 -> rating boolean
delta = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
delta2 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for r in xrange(num_train):
    delta[train_user[r]][train_business[r]] = train_rating[r] - Ubb[train_user[r]][train_business[r]]
    delta2[train_user[r]][train_business[r]] = 1


In [12]:
deltaReg = []
for i in xrange(num_user):
    X = [[0,0,0,0,0], [1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]; Y = [0,0,0,0,0,0]
    for j in xrange(num_business):
        if delta2[i][j]:
            X.append(list(DocTopDist[j]))
            Y.append(delta[i][j])
    coef = MLR(X, Y)
    deltaReg.append(np.dot(DocTopDist, coef))
    

In [13]:
row = np.array([])
col = np.array([])
val = np.array([])
TopicIn = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for i in xrange(num_user):
    for j in xrange(num_business):
        val = Ubb[i][j] + 0.8*math.atan(deltaReg[i][j])
        if val < 1: val = 1
        if val > 5: val = 5
        TopicIn[i][j] = val

TopicInPd = []
for r in xrange(num_test):
    TopicInPd.append(TopicIn[test_user[r]][test_business[r]]) 

TopicInPd_rmse = mean_squared_error(test_rating, TopicInPd)  
print TopicInPd_rmse

0.753549106386


<h1> Social Model</h1>

In [14]:
PR = {}
for u in relation:
    PR[u[0]] = len(u[1])
    
from operator import itemgetter
sorted_PR = sorted(PR.items(), key=itemgetter(1), reverse=True)

rank = {}
for u in xrange(num_user):
    rank[sorted_PR[u][0]] = u+1

Ri = []
for u in xrange(num_user):
    Ri.append(rank[u])

Wi = []
for ri in Ri:
    Wi.append(1.0/(1.0+ math.log(ri)))    


In [15]:
row = np.array([])
col = np.array([])
val = np.array([])

Rij = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
Sij = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
Tij = csr_matrix((val,(row,col)), shape=(num_user,num_user)).toarray()
Uij = csr_matrix((val,(row,col)), shape=(num_user,num_user)).toarray()

# user-item rating matrix. If ui gives a rating to vj, Rij is the rating score, otherwise 0
for r in xrange(num_train):
    Rij[train_user[r]][train_business[r]] = train_rating[r] - Ubb[train_user[r]][train_business[r]]
    Sij[train_user[r]][train_business[r]] = 1
       
# user-user social relations where Tij = 1 if ui,uj has a relation and zero otherwise
for u in relation:
    for f in u[1]:
        Tij[u[0]][f] = 1

# VIP matrix where Uij = 1 if uj is a VIP and zero otherwise        
for u in xrange(num_user):
    for v in xrange(num_user):
        if Wi[v] > vip_val:
            Uij[u][v] = 1

TR = np.dot(Tij, Rij)
TS = np.dot(Tij, Sij)
UR = np.dot(Uij, Rij)
US = np.dot(Uij, Sij)

In [16]:
Cos_norm = []
for u in xrange(num_user):
    Cos_norm.append(math.sqrt(np.dot(Rij[u], Rij[u])))

Cos = np.dot(Rij, Rij.T)
for i in xrange(num_user):
    for j in xrange(num_user):
        if Cos_norm[i] and Cos_norm[j]:
            Cos[i][j] = Cos[i][j]/Cos_norm[i]/Cos_norm[j]
        else: 
            Cos[i][j] = 0

CR = np.dot(Cos, Rij)
CS = np.dot(Cos, Sij)

In [17]:
row = np.array([])
col = np.array([])
val = np.array([])
Vij1 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
Vij2 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
Vij3 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()


for i in xrange(num_user):
    for j in xrange(num_business):
        if US[i][j]: Vij3[i][j] = lbd3*UR[i][j]/US[i][j]
        if TS[i][j]: Vij2[i][j] = lbd2*TR[i][j]/TS[i][j]
        if CS[i][j]: Vij1[i][j] = lbd1*CR[i][j]/CS[i][j]   

row = np.array([])
col = np.array([])
val = np.array([])
SocialIn1 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
SocialIn2 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
SocialIn3 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()

for i in xrange(num_user):
    for j in xrange(num_business):
        val1 = Ubb[i][j] + Vij1[i][j]
        val2 = Ubb[i][j] + Vij2[i][j]
        val3 = Ubb[i][j] + Vij3[i][j]
        
        if val1 < 1: val1 = 1
        if val1 > 5: val1 = 5
        if val2 < 1: val2 = 1
        if val2 > 5: val2 = 5
        if val3 < 1: val3 = 1
        if val3 > 5: val3 = 5
            
        SocialIn1[i][j] = val1
        SocialIn2[i][j] = val2
        SocialIn3[i][j] = val3

SocialInPd1 = []
SocialInPd2 = []
SocialInPd3 = []
for r in xrange(num_test):
    SocialInPd1.append(SocialIn1[test_user[r]][test_business[r]]) 
    SocialInPd2.append(SocialIn2[test_user[r]][test_business[r]]) 
    SocialInPd3.append(SocialIn3[test_user[r]][test_business[r]]) 

SocialInPd_rmse1 = mean_squared_error(test_rating, SocialInPd1)  
SocialInPd_rmse2 = mean_squared_error(test_rating, SocialInPd2)  
SocialInPd_rmse3 = mean_squared_error(test_rating, SocialInPd3)  

if SocialInPd_rmse1 < SocialInPd_rmse2 and SocialInPd_rmse1 < SocialInPd_rmse3:
    print "user similarity dominant rating, rmse =", SocialInPd_rmse1
    SocialInPd_rmse = SocialInPd_rmse1
elif SocialInPd_rmse2 < SocialInPd_rmse3:
    print "relation between users dominant rating, rmse =", SocialInPd_rmse2
    SocialInPd_rmse = SocialInPd_rmse2
else:
    print "VIP user dominant rating, rmse =", SocialInPd_rmse3
    SocialInPd_rmse = SocialInPd_rmse3


relation between users dominant rating, rmse = 0.769543699583


In [18]:
print "Ubb_rmse =", Ubb_rmse
print "BasicPd_rmse =", BasicPd_rmse
print "TopicInPd_rmse =", TopicInPd_rmse
print "SocialInPd_rmse =", SocialInPd_rmse

Ubb_rmse = 0.772537289011
BasicPd_rmse = 0.764162010064
TopicInPd_rmse = 0.753549106386
SocialInPd_rmse = 0.769543699583
